In [11]:
import os
import sys
sys.path.append('D:/ML/Kis_Api_test/1_DB_System/')

In [12]:
from communication import StockConnection, DB_Handle
import datetime
import pandas as pd
from queue import Queue
from tqdm import tqdm
import time

In [13]:
def date_to_string(date:datetime.date):
    conv_date = str(date)
    return conv_date[:4] + conv_date[5:7] + conv_date[-2:]

In [14]:
sc = StockConnection()
db = DB_Handle(db_name='kospi_fin')

good


In [15]:
sql = f"""drop database kospi"""
db.CUR.execute(sql)
db.commit()

In [6]:
sql = f"""create database kospi_fin"""
db.CUR.execute(sql)
db.commit()

In [50]:
kospi = pd.read_csv('../../4_SRC/current_kospi_codes.csv')
kospi['종목코드'] = kospi['종목코드'].apply(lambda x : str(x).zfill(6))

In [51]:
# kosdaq = pd.read_csv('../../4_SRC/current_kosdaq_codes.csv')
# kosdaq['종목코드'] = kosdaq['종목코드'].apply(lambda x : str(x).zfill(6))

In [52]:
kospi.iloc[358:, :].head(20)

,회사명,종목코드,상장일
358,KH필룩스,033180,2001-12-26
359,종근당바이오,063160,2001-12-11
360,신한지주,055550,2001-09-10
361,코웨이,021240,2001-08-07
362,세아홀딩스,058650,2001-07-30
363,LG생활건강,051900,2001-04-25
364,LG화학,051910,2001-04-25
365,대우건설,047040,2001-03-23
366,포스코인터내셔널,047050,2001-03-23
367,대우조선해양,042660,2001-02-02


In [53]:
for idx, context in tqdm(kospi.iloc[358:, :].iterrows()):
    if True:
        db.create_table(context['회사명'])
        print(context['회사명'])
        first_y, first_m, first_d = context['상장일'].split('-')
        first_date = datetime.date(int(first_y), int(first_m), int(first_d))
        start = first_date
        end = start + datetime.timedelta(99)
        while end <= datetime.date.today():
#             print(f'st:{start}, end:{end}')
            result = sc.date_price(no=context['종목코드'], 
                                start=date_to_string(start), 
                                end=date_to_string(end), 
                                period='D')
            for idx, res in enumerate(result['output2'][::-1]):
                try:
                    buf = []
                    buf.append(result['output1']['stck_shrn_iscd'])
                    buf.append(res['stck_bsop_date'])
                    buf.append(res['stck_clpr'])
                    buf.append(res['stck_oprc'])
                    buf.append(res['stck_hgpr'])
                    buf.append(res['stck_lwpr'])
                    buf.append(res['acml_vol'])
                    buf.append(res['acml_tr_pbmn'])
                    db.insert(table_name=context['회사명'], telegram=buf)
                except:
                    pass
            time.sleep(1.01)
            start = end + datetime.timedelta(1)
            end = start + datetime.timedelta(99)
#         print(f'st:{start}, end:{end}')
        result = sc.date_price(no=context['종목코드'], 
                                start=date_to_string(start), 
                                end=date_to_string(datetime.date.today()), 
                                period='D')
        try:
            for idx, res in enumerate(result['output2'][::-1]):
                buf = []
                buf.append(result['output1']['stck_shrn_iscd'])
                buf.append(res['stck_bsop_date'])
                buf.append(res['stck_clpr'])
                buf.append(res['stck_oprc'])
                buf.append(res['stck_hgpr'])
                buf.append(res['stck_lwpr'])
                buf.append(res['acml_vol'])
                buf.append(res['acml_tr_pbmn'])
                db.insert(table_name=context['회사명'], telegram=buf)
        except:
            pass
    
    else:
        pass


0it [00:00, ?it/s]

KH필룩스


1it [01:34, 94.22s/it]

종근당바이오


2it [03:08, 94.39s/it]

...

웰바이오텍


43it [1:14:25, 113.82s/it]

한국주강


44it [1:16:21, 114.55s/it]

덕양산업


In [ ]:
#한국금융지주